# New Label 

Cách 1: 

- nhóm Intermediate: bethesda 1; 
- nhóm high probability of benign (HPB): bethesda 2, 3, và 4; 
- nhóm high probability of malignant (HPM): bethesda 5 và 6

# New Label 

Cách 2: 
- nhóm Intermediate: bethesda 1, 3, và 4; 
- nhóm high probability of benign (HPB): bethesda 2; 
- nhóm high probability of malignant (HPM): bethesda 5 và 6


# PREPARE CSV FILE

In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv('../data/Tanzania-Data/TAN_THYROID_summary.csv', index_col=0)
df

,Scan ID,Cytology no.,Diagnosis/Bethesda system,Bethesda actual,Histopathology,Present,Present (Manual)
0,TAN001,C-11-22,Malignant,5,Anaplastic carcinoma,1,exist
1,TAN002,C-20-22,Benign nodule,1,Simple Nodular Goitre,1,exist
2,TAN003,C-53-23,Benign nodule,1,Simple Nodular Goitre,1,exist
3,TAN004,C-36-23,Benign nodule,1,Simple Nodular Goitre,1,exist
4,TAN005,C-373-20,Benign nodule,1,Simple Nodular Goitre,1,exist
...,...,...,...,...,...,...,...
100,TAN101,C-076-23,Carcinoma,5,No biopsy,1,exist
101,TAN102,C-285-23,Goitre/thyroiditis,1,Cystic goitre,1,exist
102,TAN103,CP-01-23,Carcinoma,5,Anaplastic carcinoma,1,exist
103,TAN104,C-302-23,Colloid goitre,1,Colloid goitre,1,exist


In [3]:
# opt2
def get_label(x):
    if (x==0) or (x==2) or (x==3):
        return 0
    elif x==1:
        return 1
    else:
        return 2

In [4]:
df['label'] = df['Bethesda actual'].apply(lambda x: get_label(x))

In [5]:
df['Bethesda actual'].value_counts().sort_index()

Bethesda actual
0     1
1    78
2     2
3    11
4     1
5    12
Name: count, dtype: int64

In [6]:
df['label'].value_counts().sort_index()

label
0    14
1    78
2    13
Name: count, dtype: int64

In [7]:
df.to_csv('../metadata/TAN_groundtruth.csv')

Prepare Train/Val/Test dataset

In [8]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df['label'], test_size=0.3)

In [9]:
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
skf = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
skf.get_n_splits(df['label'],df['label'])
for i, (train_index, test_index) in enumerate(skf.split(df['label'],df['label'])):
    df.loc[train_index,f'fold_{i}'] = 'train'
    df.loc[test_index,f'fold_{i}'] = 'test'

In [10]:
import re
df['patient_id'] = df['Scan ID'].apply(lambda x: int(re.findall(r'TAN(\d+)',x)[0]))

In [11]:
df

,Scan ID,Cytology no.,Diagnosis/Bethesda system,Bethesda actual,Histopathology,Present,Present (Manual),label,fold_0,fold_1,fold_2,fold_3,fold_4,patient_id
0,TAN001,C-11-22,Malignant,5,Anaplastic carcinoma,1,exist,2,test,train,train,train,train,1
1,TAN002,C-20-22,Benign nodule,1,Simple Nodular Goitre,1,exist,1,train,train,test,train,train,2
2,TAN003,C-53-23,Benign nodule,1,Simple Nodular Goitre,1,exist,1,train,train,test,train,train,3
3,TAN004,C-36-23,Benign nodule,1,Simple Nodular Goitre,1,exist,1,train,train,train,train,train,4
4,TAN005,C-373-20,Benign nodule,1,Simple Nodular Goitre,1,exist,1,train,test,test,train,train,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,TAN101,C-076-23,Carcinoma,5,No biopsy,1,exist,2,train,train,train,train,train,101
101,TAN102,C-285-23,Goitre/thyroiditis,1,Cystic goitre,1,exist,1,train,train,train,train,train,102
102,TAN103,CP-01-23,Carcinoma,5,Anaplastic carcinoma,1,exist,2,train,train,train,train,train,103
103,TAN104,C-302-23,Colloid goitre,1,Colloid goitre,1,exist,1,train,train,train,train,test,104


In [12]:
df.to_csv('../metadata/TAN_patient_dataset.csv')

Read image files

In [13]:
import os
import pathlib

In [14]:
p = pathlib.Path('../data/Tanzania-Data')
all_folders = list(p.iterdir())
all_folders = [i for i in all_folders if i.is_dir()]
all_folders[:3]

[PosixPath('../data/Tanzania-Data/TAN097'),
 PosixPath('../data/Tanzania-Data/TAN010'),
 PosixPath('../data/Tanzania-Data/TAN009')]

In [15]:
rows = []
for f in all_folders:
    if 'TAN' in f.name:
        patient_id = int(re.findall(r'TAN(\d+)',f.name)[0])
        all_imgs = list(f.iterdir())
        all_imgs = [i for i in all_imgs if i.name.startswith('IMG') and i.name.endswith('.jpg')]
        for img in all_imgs:
            rows.append((patient_id, str(img)))

In [16]:
image_dataset = pd.DataFrame(rows, columns=['patient_id', 'image_path'])
image_dataset

,patient_id,image_path
0,97,../data/Tanzania-Data/TAN097/IMG_2301251206156...
1,97,../data/Tanzania-Data/TAN097/IMG_2301251202141...
2,97,../data/Tanzania-Data/TAN097/IMG_2301251158260...
3,97,../data/Tanzania-Data/TAN097/IMG_2301251200592...
4,97,../data/Tanzania-Data/TAN097/IMG_2301251204265...
...,...,...
1192,101,../data/Tanzania-Data/TAN101/IMG_2302021504226...
1193,101,../data/Tanzania-Data/TAN101/IMG_2302021458365...
1194,101,../data/Tanzania-Data/TAN101/IMG_2302021505018...
1195,101,../data/Tanzania-Data/TAN101/IMG_2302021505310...


In [17]:
image_dataset.to_csv('../metadata/TAN_image_path.csv')